In [1]:
pip install faker

In [2]:
import sqlite3
from faker import Faker
import random
from datetime import datetime

# Crear una instancia de Faker
fake = Faker()

# Conectar a la base de datos SQLite (se creará automáticamente)
conn = sqlite3.connect('tienda_minorista.db')
cursor = conn.cursor()

# Crear tabla de Ventas sin la columna Competitor_Price
cursor.execute('''CREATE TABLE IF NOT EXISTS Ventas (
                    Customer_Id INTEGER,
                    DateTime TEXT,
                    Customer_Name TEXT,
                    Customer_Category TEXT,
                    Marital_Status TEXT,
                    Age INTEGER,
                    Gender TEXT,
                    Products TEXT,
                    Total_Items INTEGER,
                    Total_Price REAL,
                    Payment_Method TEXT,
                    State TEXT,
                    Season TEXT,
                    PCL TEXT)''')

# Lista de categorías de clientes y rangos de edad
customer_categories = ['Student', 'Professional', 'Homemaker', 'Teenager', 'Young', 'Adult', 'Elderly']
age_ranges = {'Student': (14, 17), 'Professional': (18, 29), 'Homemaker': (30, 64), 'Teenager': (14, 17), 'Young': (18, 29), 'Adult': (30, 64), 'Elderly': (65, 100)}

# Lista de géneros
genders = ['Male', 'Female', 'Other']

# Lista de productos de comida en inglés
food_products = ['Rice', 'Beans', 'Chicken', 'Fish', 'Salad', 'Soup', 'Pasta', 'Burger', 'Pizza', 'Tacos',
                 'Steak', 'Sushi', 'Sandwich', 'Fries', 'Curry', 'Burrito', 'Lasagna', 'Hotdog', 'Ramen', 'Donut',
                 'Eggs', 'Milk', 'Bread', 'Cheese', 'Yogurt', 'Apple', 'Banana', 'Orange', 'Broccoli', 'Carrot',
                 'Tomato', 'Cucumber', 'Lettuce', 'Potato', 'Onion', 'Garlic', 'Sausage', 'Ham', 'Bacon', 'Salami']

# Lista de estados de USA
us_states = [
    'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida',
    'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine',
    'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
    'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
    'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
    'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'
]

# Pesos para cada estado (proporciones aproximadas)
state_weights = [5, 1, 7, 4, 50, 4, 3, 1, 23, 10, 1, 10, 5, 3, 2, 4, 4, 1, 2, 3, 7, 3, 4, 3, 3, 1, 1, 2, 2, 3, 4, 5, 10, 4, 2, 5, 1, 6, 3, 3, 1, 2, 2, 7, 40, 3, 1, 9, 2, 1]

# Generar 200,000 registros de datos aleatorios para la tabla de Ventas
for _ in range(200000):
    Customer_Id = random.randint(1000, 9999)
    DateTime = fake.date_time_between(start_date='-1y', end_date='now').strftime('%Y-%m-%d %H:%M:%S')
    Customer_Name = fake.first_name() + ' ' + fake.last_name()

    # Seleccionar aleatoriamente una categoría de cliente
    customer_category = random.choice(customer_categories)

    # Generar estado civil para las categorías de cliente
    if customer_category in ['Student', 'Teenager']:
        Marital_Status = 'Single'
    elif customer_category == 'Young':
        Marital_Status = random.choices(['Single', 'Married'], weights=[0.5, 0.5])[0]
    else:
        Marital_Status = fake.random_element(elements=('Single', 'Married', 'Divorced', 'Widowed'))

    # Generar una edad aleatoria dentro del rango correspondiente a la categoría de cliente
    age_range = age_ranges[customer_category]
    Age = random.randint(age_range[0], age_range[1])

    # Generar una lista de productos de comida aleatorios en inglés con sus cantidades
    products = [random.choice(food_products) for _ in range(random.randint(1, 5))]  # Generar entre 1 y 5 productos aleatorios
    total_items = sum([random.randint(1, 10) for _ in products])  # Sumar las cantidades de los productos

    Total_Price = round(random.uniform(10.0, 500.0), 2)

    # Modificar la lista de métodos de pago
    payment_methods = ['Credit Card', 'Debit Card', 'Cash', 'Mobile Payment']
    Payment_Method = random.choice(payment_methods)

    # Seleccionar un estado aleatorio con pesos específicos
    State = random.choices(us_states, weights=state_weights)[0]
    #State = random.choices(us_states, weights=state_weights)


    Season = random.choice(['Spring', 'Summer', 'Autumn', 'Winter'])

    # Generar valor para la columna PCL (Programa de Fidelización de Cliente)
    PCL = random.choice(['True', 'False'])

    # Elegir el género aleatoriamente, con probabilidades ajustadas
    gender_probabilities = [0.35, 0.55, 0.1]  # Male, Female, Other
    Gender = random.choices(genders, weights=gender_probabilities)[0]

    cursor.execute('''INSERT INTO Ventas (Customer_Id, DateTime, Customer_Name, Customer_Category, Marital_Status, Age, Gender, Products, Total_Items, Total_Price, Payment_Method, State, Season, PCL)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                   (Customer_Id, DateTime, Customer_Name, customer_category, Marital_Status, Age, Gender, ', '.join(products), total_items, Total_Price, Payment_Method, State, Season, PCL))

# Guardar cambios y cerrar conexión
conn.commit()
conn.close()

print("Base de datos creada exitosamente con 200,000 registros en la tabla de Ventas.")



Base de datos creada exitosamente con 200,000 registros en la tabla de Ventas.


In [3]:
import sqlite3
import pandas as pd

# Conectar a la base de datos SQLite
conn = sqlite3.connect('tienda_minorista.db')

# Consultar la tabla de Ventas y cargar los datos en un DataFrame de pandas
df = pd.read_sql_query("SELECT * FROM Ventas", conn)

# Guardar el DataFrame en un archivo CSV
df.to_csv('DM_Sales.csv', index=False)

# Cerrar la conexión a la base de datos
conn.close()

print("Archivo CSV generado exitosamente.")

Archivo CSV generado exitosamente.
